In [20]:
import os
from IPython.display import HTML
from IPython import display
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_petri_model,
    setup_petri_model,
    sample
)
import numpy as np
from typing import Iterable
from pyciemss.utils.interface_utils import (
    assign_interventions_to_timepoints,
    interventions_and_sampled_params_to_interval,
    convert_to_output_format
)
from pyciemss.utils import get_tspan
import matplotlib.pyplot as plt
import torch
from torch import tensor

Plotting utils

In [7]:
COLORS=['red','green','blue','black','gold','purple','orangered']
# TODO: Add plot title, axis labels

# def plot_allsolns(samples_df, title):
#     QoIs = ["D_sol", "E_sol", "I_sol", "R_sol", "S_sol"]
#     QoI_samples = {key: None for key in QoIs}
#     for QoI in QoIs:
#         # Aggregate results
#         total_pop = samples_df[samples_df["sample_id"] == 0]["S_sol"].to_numpy()[0]
#         QoI_samples[QoI] = [samples_df[samples_df["sample_id"] == i][QoI].to_numpy()/total_pop * 100 for i in range(num_samples)]
#     # Plot median of all QoIs
#     fig, ax = plt.subplots()
#     for i, QoI in enumerate(QoIs):
#         ax.plot(timepoints, np.percentile(QoI_samples[QoI], 50, axis=0), color=COLORS[i], label=QoI)
#         ax.set_title(title)
#         ax.legend()
def aggregate_results(samples_df, num_samples=1):
    QoIs = ["D_sol", "E_sol", "I_sol", "R_sol", "S_sol"]
    all_QoIs = {key: None for key in QoIs}
    for QoI in QoIs:
        # Aggregate results
        total_pop = samples_df[samples_df["sample_id"] == 0]["S_sol"].to_numpy()[0]
        all_QoIs[QoI] = [samples_df[samples_df["sample_id"] == i][QoI].to_numpy()/total_pop * 100 for i in range(num_samples)]
    return all_QoIs

In [8]:
def plotseird(t, S, E, I, R, D=None, L=None, R0=None, Alpha=None, CFR=None):
    f, ax = plt.subplots(1,1,figsize=(10,4))
    ax.plot(t, (S/N)*100, 'b--', alpha=0.7, linewidth=2, label='Susceptible')
    ax.plot(t, (E/N)*100, 'y.', alpha=0.7, linewidth=2, label='Exposed')
    ax.plot(t, (I/N)*100, 'r', alpha=0.7, linewidth=2, label='Infected')
    ax.plot(t, (R/N)*100, 'g-.', alpha=0.7, linewidth=2, label='Recovered')
  
    if D is not None:
        ax.plot(t, D/N, 'k', alpha=0.7, linewidth=2, label='Dead')
        ax.plot(t, ((S+E+I+R+D)/N)*100, 'c:', alpha=0.7, linewidth=2, label='Total')
    else:
        ax.plot(t, ((S+E+I+R)/N)*100, 'c:', alpha=0.7, linewidth=2, label='Total')

    ax.set_xlabel('Time (days)')
    ax.set_ylabel('Percentage of Population (%)')

    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend(borderpad=2.0)
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    if L is not None:
        plt.title("Lockdown after {} days".format(L))
    plt.show();

    if R0 is not None or CFR is not None:
       f = plt.figure(figsize=(12,4))
    
    if R0 is not None:
        # sp1
        ax1 = f.add_subplot(121)
        ax1.plot(t, R0, 'b--', alpha=0.7, linewidth=2, label='R_0')

        ax1.set_xlabel('Time (days)')
        ax1.title.set_text('R_0 over time')
        # ax.set_ylabel('Number (1000s)')
        # ax.set_ylim(0,1.2)
        ax1.yaxis.set_tick_params(length=0)
        ax1.xaxis.set_tick_params(length=0)
        ax1.grid(b=True, which='major', c='w', lw=2, ls='-')
        legend = ax1.legend()
        legend.get_frame().set_alpha(0.5)
        for spine in ('top', 'right', 'bottom', 'left'):
            ax.spines[spine].set_visible(False)

    if Alpha is not None:
        # sp2
        ax2 = f.add_subplot(122)
        ax2.plot(t, Alpha, 'r--', alpha=0.7, linewidth=2, label='alpha')

        ax2.set_xlabel('Time (days)')
        ax2.title.set_text('fatality rate over time')
        # ax.set_ylabel('Number (1000s)')
        # ax.set_ylim(0,1.2)
        ax2.yaxis.set_tick_params(length=0)
        ax2.xaxis.set_tick_params(length=0)
        ax2.grid(b=True, which='major', c='w', lw=2, ls='-')
        legend = ax2.legend()
        legend.get_frame().set_alpha(0.5)
        for spine in ('top', 'right', 'bottom', 'left'):
            ax.spines[spine].set_visible(False)

    plt.show()

# SCENARIO 1: Forecasting with non-pharmaceutical interventions (NPIs) such as, masking and social distancing. 

**"You want to implement a masking intervention in a simple compartmental model and simulate epidemic trajectories under different compliance scenarios. You found an existing model that incorporates masking as a time-dependent modification to the $\beta$ parameter, and you want to ensure that the model is working as expected by reproducing plots in the publication"**

### PART 1
**(A)** TA1/TA2 provide the AMR for a COVID-19 model with compartments SEIRD (Susceptible, Exposed, Infected, Recovered, Death) and, importantly, a time-dependent transmission rate $\beta$. A mask order may be implemented $t_0$ days after the first case is detected (at time $t = 0$), and $\beta$ varies in time according to the effectiveness of the NPI under various levels of compliance.\
Transmission rate $\beta(t)$ is defined as, \begin{align} \beta(t) &= \kappa m(t) \\
m(t) &= \beta_c + \frac{\beta_s - \beta_c}{1 + e^{-k (t_0 - t)}},
\end{align}
where $\kappa$ is an arbitrary constant, and $m(t)$ represents adoption of a mask order $t_0$ days after the first case is detected, assuming 100% compliance. \
Model Source: https://doi.org/10.3390/ijerph18179027 \
AMR location: https://github.com/indralab/mira/blob/hackathon/notebooks/hackathon_2023.07/scenario1_a.json 

Model Schematic:

<div>
<img src="https://i.imgur.com/GXshSZH.png" width="600" align="left"/>
</div>

In [9]:
DEMO_PATH = "scenario1.ipynb"

SEIRD_model_path = "scenario1_a.json"

### TA3 TASK 1:
**(B)** Perform model simulation with various delays ($t_0$ values) in mask order implementation, and reproduce Fig. 3, including peak prevalence estimates. 

Figure 3 from the paper
<div>
<img src="https://i.imgur.com/u41OiB6.png" width="500" align="left"/>
</div>

Simulation Parameters

In [10]:
num_samples = 1
start_time = 0
end_time = 365
num_timepoints = (end_time-start_time)*10 + 1
timepoints = get_tspan(start_time, end_time, num_timepoints)

Delayed Masking Implementation

Modifying modified logistic function ($m(t)$ in the paper)

Used to model this transition by setting the infection rate before and after implementing mask mandate assuming 100% compliance.

In [11]:
# def modify_logistic_func(t, t0=0, k=1, beta_c=0.4, beta_s=1):
#     return beta_c + (beta_s-beta_c)/(1+np.exp(-1*k*(t0-t)))

Acquire array of betas for intervention according to  $\beta(t) = \kappa m(t)$

Note: In order to intervene on a distribution of $\beta$ values, randomly sampled values of $\beta_s$ can be passed in respectively

In [12]:
# def get_intervention_betas(start_intervene_day, start_day, end_day, kappa=1, beta_s=1, beta_c=0.4):
#     intervention_time = np.arange(start_day,end_day+1) + 1e-6
#     m = [modify_logistic_func(t,t0=start_intervene_day) for t in intervention_time]
#     intervened_beta_values = kappa*m
#     return [(intervention_time[i], 'beta_s', intervened_beta_values[i]) for i in range(len(intervention_time))]

Reproduction of figure 1 from the paper

In [13]:
# intervention_time = np.arange(start_time,end_time+1)
# m = [modify_logistic_func(t,t0=89) for t in intervention_time]
# plt.plot(intervention_time, m)

#### Figure 3 Recreation

Case A - 0 Days Delay of Mask Mandate

In [14]:
intervened_t0 = [(1e-6,'t_0',0)]
implement_0day_samples = load_and_sample_petri_model(
    SEIRD_model_path, num_samples, timepoints=timepoints, interventions=intervened_t0
)

Case B - 50 Days Delay of Mask Mandate

In [15]:
intervened_t0 = [(50+1e-6,'t_0',50)]
implement_50day_samples = load_and_sample_petri_model(
    SEIRD_model_path, num_samples, timepoints=timepoints, interventions=intervened_t0
)

Case C - 100 Days Delay of Mask Mandate

In [16]:
intervened_t0 = [(100+1e-6,'t_0',100)]
implement_100day_samples = load_and_sample_petri_model(
    SEIRD_model_path, num_samples, timepoints=timepoints, interventions=intervened_t0
)

Case D - Control (no intervention)

In [18]:
control_t0 = [(365+1e-6,'t_0',365)] # NOTE: I'm not sure if this will overlap with pre-existing simulation timepoint.
control_samples = load_and_sample_petri_model(
    SEIRD_model_path, num_samples, timepoints=timepoints, interventions=intervened_t0)


TypeError: getattr(): attribute name must be string

In [ ]:
control_samples

Plot results and recreate Figure 3

In [ ]:
QoIs_0day = aggregate_results(implement_0day_samples, num_samples)
QoIs_50day = aggregate_results(implement_50day_samples, num_samples)
QoIs_100day = aggregate_results(implement_100day_samples, num_samples)
QoIs_control = aggregate_results(control_samples, num_samples)

# ["D_sol", "E_sol", "I_sol", "R_sol", "S_sol"]
# plotseird(t, S, E, I, R, D=None, L=None, R0=None, Alpha=None, CFR=None)
plotseird(timepoints, QoIs_0day["S_sol"], QoIs_0day["E_sol"], QoIs_0day["I_sol"], QoIs_0day["R_sol"])
plotseird(timepoints, QoIs_50day["S_sol"], QoIs_50day["E_sol"], QoIs_50day["I_sol"], QoIs_50day["R_sol"])
plotseird(timepoints, QoIs_100day["S_sol"], QoIs_100day["E_sol"], QoIs_100day["I_sol"], QoIs_100day["R_sol"])
plotseird(timepoints, QoIs_control["S_sol"], QoIs_control["E_sol"], QoIs_control["I_sol"], QoIs_control["R_sol"])

In [ ]:
# # Plot fill-between of all QoIs
# fig, ax = plt.subplots()
# for i, QoI in enumerate(QoIs):
#     ax.fill_between(timepoints, np.percentile(QoI_samples[QoI], 25, axis=0), np.percentile(QoI_samples[QoI], 75, axis=0), color=COLORS[i], alpha=0.5)
#     ax.plot(timepoints, np.percentile(QoI_samples[QoI], 50, axis=0), color=COLORS[i], label=QoI)
#     ax.legend()

In [ ]:
# # Plot median of all QoIs
# fig, ax = plt.subplots()
# for i, QoI in enumerate(QoIs):
#     ax.plot(timepoints, np.percentile(QoI_samples[QoI], 50, axis=0), color=COLORS[i], label=QoI)
#     ax.legend()

**(C)** TA2 updates the AMR to redefine $\beta(t)$ as, \begin{align} \beta(t) &= \kappa m(t) \\
m(t) &= \beta_{nc} + \frac{\beta_s - \beta_c}{1 + e^{-k_1(t_0 - t)}} + \frac{\beta_c - \beta_{nc}}{1 + e^{-k_2(t_1 - t)}}, \end{align}
to model gradual non-compliance with a mask order. \
AMR location: https://github.com/indralab/mira/blob/hackathon/notebooks/hackathon_2023.07/scenario1_c.json 

**TA3 TASK 2:** Perform model simulation over a range of delays in mask enforcement, and plot the peak prevalence to reproduce Fig. 5.
<div>
<img src="https://i.imgur.com/dDQX87i.png" width="500" align="left"/>
</div>

**(D)** TA2 updates AMR of SEIRD model to include possibility for reinfection, producing the SEIRS model.
AMR location: https://github.com/indralab/mira/blob/hackathon/notebooks/hackathon_2023.07/scenario1_d.json \
Model Schematic:
<div>
<img src="https://i.imgur.com/KMgRtbO.png" width="300" align="left"/>
</div>

**TA3 TASK 3:** Perform model simulation and again reproduce Fig. 5 (as in TASK 2). Compare with previous results (as in 1C), "what impact does immunity loss and potential for reinfection have?"

**(E) TA3 TASK 4:** Repeat the previous SEIRS model simulation (as in TASK 3), but add *uncertainty* by assessing a range of non-compliance levels (as defined by $\beta_{nc}$ and $k_2$) and reinfection rates (as controlled through $\epsilon$). Plot the state variables over time along with the uncertainty ranges.

### PART 2
**(D) TA3 Recommended TASK 5:** Find a real-world example of a mask order being implemented. Calibrate the model from Part 1A using data from the COVID-19 ForecastHub for a period before the mask intervention is implemented, and forecast through the intervention period. Demonstrate how the forecast varies based on compliance rates.